In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt
/kaggle/input/movie-content-based-filtering-recommender/data/content_item_vecs.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_movie_list.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_y_train.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train_header.txt
/kaggle/input/movie-content-based-filtering-recommender/data/content_top10_df.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_to_genre.pickle
/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_bygenre_df.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train.csv


In [2]:
from numpy import genfromtxt
import csv
from collections import defaultdict
import pickle
import tabulate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import Model
from keras.layers import Dense, Input, Lambda, Dot
from keras.losses import MeanSquaredError
from keras.optimizers import Adam
from tensorflow.linalg import l2_normalize
import keras.backend as K
import numpy.ma as ma

In [3]:
RANDOM_STATE = 1

In [4]:
top10_df = pd.read_csv("/kaggle/input/movie-content-based-filtering-recommender/data/content_top10_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.106061,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.021277,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.118919,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.867647,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.238255,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.778523,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.066434,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,3.960993,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.871212,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.160305,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [5]:
bygenre_df = pd.read_csv("/kaggle/input/movie-content-based-filtering-recommender/data/content_bygenre_df.csv")
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.37,10377
1,Adventure,234,3.42,8785
2,Animation,76,3.63,2588
3,Children,69,3.44,2472
4,Comedy,326,3.36,8911
5,Crime,139,3.54,4671
6,Documentary,13,3.81,280
7,Drama,342,3.61,10201
8,Fantasy,124,3.37,4468
9,Horror,56,3.20,1345


In [6]:
def load_data():
    item_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train.csv", delimiter=",")
    user_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train.csv", delimiter=",")
    y_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_y_train.csv", delimiter=",")
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt", newline="") as f:
        item_features = list(csv.reader(f))[0]
        
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train_header.txt", newline="") as f:
        user_features = list(csv.reader(f))[0]
    
    item_vecs = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_vecs.csv", delimiter = ",")
    movie_dict = defaultdict(dict)
    count = 0 
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_movie_list.csv", newline="") as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count+=1
            else:
                count+=1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]
    
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_to_genre.pickle", 'rb') as f:
        user_to_genre = pickle.load(f)
    
    return item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre
    
# file = "/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt"
# with open(file, 'r') as file:
#     content = file.read()
# content

In [7]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3   # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1   # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

print("No of training vectors: ",len(item_train))
item_train[:2]

No of training vectors:  50884


array([[6.87400000e+03, 2.00300000e+03, 3.96183206e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [8.79800000e+03, 2.00400000e+03, 3.76136364e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00]])

In [8]:
def print_train(X_train, features, vs, u_s, maxcount=5, user=True):
    if user:
        flist = [".0f", ".0f", ".1f",
                 ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f"]
    else:
        flist = [".0f", ".0f", ".1f", 
                 ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f"]

    head = features[:vs]
    if vs<u_s:
        print(f"Error, vector start {vs} should be greater than user start {u_s}")

    for i in range(u_s):
        head[i] = "[" + head[i] + "]"
        
    genres = features[vs:]
    hdr = head + genres
    disp = [split_str(hdr, 5)]
    count = 0
    
    for i in range(0, X_train.shape[0]):
        if count == maxcount: break
        count += 1
        disp.append([X_train[i, 0].astype(int),
                     X_train[i, 1].astype(int),
                     X_train[i, 2].astype(float),
                     *X_train[i, 3:].astype(float)
                    ])
    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=flist, numalign='center')
    return table

In [9]:
def split_str(ifeatures, smax):
    ''' split the feature name strings to tables fit '''
    ofeatures = []
    for s in ifeatures:
        if not ' ' in s:  # skip string that already have a space
            if len(s) > smax:
                mid = int(len(s)/2)
                s = s[:mid] + " " + s[mid:]
        ofeatures.append(s)
    return ofeatures
    

In [10]:
print_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [11]:
print_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [12]:
y_train.shape

(50884,)

In [13]:
user_train

array([[  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       [  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       [  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       ...,
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ],
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ],
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ]])

In [14]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1,1))
scalerTarget.fit(y_train.reshape(-1,1))
y_train = scalerTarget.transform(y_train.reshape(-1,1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [15]:
print_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6


In [16]:
item_train, item_test = train_test_split(item_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)
user_train, user_test = train_test_split(user_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)
y_train, y_test       = train_test_split(y_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)

In [17]:
print("User Training:", user_train.shape)
print("User Test:", user_test.shape)

print("Item Training:", item_train.shape)
print("Item Test:", item_test.shape)

User Training: (40707, 17)
User Test: (10177, 17)
Item Training: (40707, 17)
Item Test: (10177, 17)


In [18]:
num_outputs = 32
tf.random.set_seed(RANDOM_STATE)

user_NN = Sequential([
    Dense(units=256, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=32, activation='linear'),
])

item_NN = Sequential([
    Dense(units=256, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=32, activation='linear'),
])

input_user = Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = Lambda(lambda x: l2_normalize(x, axis=1))(vu)

input_item = Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = Lambda(lambda x: l2_normalize(x, axis=1))(vm)

output = Dot(axes=1)([vu, vm])

model = Model([input_user, input_item], output)
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 14)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 16)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 32)             │         40,864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_1 (Sequential) │ (None, 32)             │         41,376 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 32)             │              0 │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1 (Lambda)         │ (None, 32)             │              0 │ sequential_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dot (Dot)                 │ (None, 1)              │              0 │ lambda[0][0],          │
│                           │                        │                │ lambda_1[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
cost_fn = MeanSquaredError()
opt = Adam(learning_rate = 0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [20]:
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.1297
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1140
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1076
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1039
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1010
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0988
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0968
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0947
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0927
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0909
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0893
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0878
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0865
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0852
Epoch 15/30
1273/1273 ━━━━━━━

In [21]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0877


0.08373530209064484

In [22]:
user_test

array([[ 0.54027243, -1.00604417,  1.84452438, ...,  1.39257084,
         1.25923081,  2.06047934],
       [ 0.86630019, -0.48554484,  1.51054443, ...,  1.30913427,
         1.11423081,  1.3013332 ],
       [-1.30909191, -1.0580941 ,  1.94889311, ...,  0.69726612,
         1.5202308 ,  1.6566782 ],
       ...,
       [-0.94933714, -1.01905665,  0.42510958, ...,  0.69726612,
         0.21523084,  0.38066661],
       [-1.70257369,  0.25616671,  0.11200338, ...,  0.34961376,
         0.24423084,  0.34836252],
       [ 0.33228921,  1.31017785,  0.32074085, ...,  0.02977359,
         0.59223083,  0.21914616]])

In [23]:
user_test[:, u_s:]

array([[ 1.71835678,  1.74795164,  1.5550193 , ...,  1.39257084,
         1.25923081,  2.06047934],
       [ 1.32949916,  1.41409604,  0.61849708, ...,  1.30913427,
         1.11423081,  1.3013332 ],
       [ 1.8536116 ,  2.02909321, -3.76612966, ...,  0.69726612,
         1.5202308 ,  1.6566782 ],
       ...,
       [-0.12449018, -0.07946853,  1.02290441, ...,  0.69726612,
         0.21523084,  0.38066661],
       [ 0.09529891,  0.14895899,  0.26730125, ...,  0.34961376,
         0.24423084,  0.34836252],
       [ 0.43343597,  0.4476719 ,  0.75684696, ...,  0.02977359,
         0.59223083,  0.21914616]])

In [24]:
print_train(user_train[:, u_s:], user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery
0,0,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,0,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
0,0,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,0,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
0,0,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


**Predictions**

**Predictions for a New User**

In [25]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

In [26]:
user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

user_vec

array([[5.e+03, 3.e+00, 0.e+00, 0.e+00, 5.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00, 0.e+00, 5.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00]])

In [27]:
def gen_user_vecs(user_vec, num_items):
    user_vecs = np.tile(user_vec, (num_items,1))
    return user_vecs

In [28]:
(item_vecs)

array([[4.05400000e+03, 2.00100000e+03, 2.84375000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.06900000e+03, 2.00100000e+03, 2.90909091e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.14800000e+03, 2.00100000e+03, 2.93589744e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.77765000e+05, 2.01700000e+03, 3.53846154e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.79819000e+05, 2.01700000e+03, 3.12500000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.87593000e+05, 2.01800000e+03, 3.87500000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [29]:
user_vecs = gen_user_vecs(user_vec, len(item_vecs))

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

y_p = model.predict([suser_vecs[:,u_s:], sitem_vecs[:,i_s:]])

y_pu = scalerTarget.inverse_transform(y_p)
# y_p

sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()
sorted_ypu = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [30]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

In [31]:
print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

y_p,movie id,rating ave,title,genres
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.2,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.2,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.2,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.2,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.1,4896,3.8,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),Adventure|Children|Fantasy
4.1,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.1,118696,3.4,The Hobbit: The Battle of the Five Armies (2014),Adventure|Fantasy
4.1,54259,3.6,Stardust (2007),Adventure|Comedy|Fantasy|Romance
4,108932,3.9,The Lego Movie (2014),Action|Adventure|Animation|Children|Comedy|Fantasy


**Predictions for an existing user**

In [32]:
uid = 2

In [33]:
def get_user_vecs(user_id, user_train, item_vecs,  user_to_genre):
    if user_id not in user_to_genre:
        print("Error: User ID nott found!")
        return

    user_vec_found = False

    for i in range(len(user_train)):
        if user_train[i,0] == user_id:
            user_vec = user_train[i]
            user_vec_found = True
            break
    if not user_vec_found:
        print("Error: Did not find User ID in user_train")

    num_items = len(item_vecs)
    user_vecs = np.tile(user_vec, (num_items, 1))

    y = np.zeros(num_items)
    for i in range(num_items):
        movie_id = item_vecs[i,0]
        if movie_id in user_to_genre[user_id]['movies']:
            y[i] = user_to_genre[user_id]['movies'][movie_id]
        else:
            y[i] = 0
    return user_vecs, y
    

In [34]:
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [35]:
def print_existing_user(y_p, y, user, items, ivs, uvs, movie_dict, maxcount=10):
    """ print results of prediction for a user who was in the database.
        Inputs are expected to be in sorted order, unscaled.
    """
    count = 0
    disp = [["y_p", "y", "user", "user genre ave", "movie rating ave", "movie id", "title", "genres"]]
    count = 0
    for i in range(0, y.shape[0]):
        if y[i, 0] != 0:  # zero means not rated
            if count == maxcount:
                break
            count += 1
            movie_id = items[i, 0].astype(int)

            offsets = np.nonzero(items[i, ivs:] == 1)[0]
            genre_ratings = user[i, uvs + offsets]
            disp.append([y_p[i, 0], y[i, 0],
                         user[i, 0].astype(int),      # userid
                         np.array2string(genre_ratings, 
                                         formatter={'float_kind':lambda x: "%.1f" % x},
                                         separator=',', suppress_small=True),
                         items[i, 2].astype(float),    # movie average rating
                         movie_id,
                         movie_dict[movie_id]['title'],
                         movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".1f"])
    return table

In [36]:
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.6,5.0,2,[4.0],4.3,80906,Inside Job (2010),Documentary
4.4,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
4.3,3.5,2,"[4.0,4.1,4.0,3.9]",3.8,8798,Collateral (2004),Action|Crime|Drama|Thriller
4.3,4.0,2,"[4.0,4.1,3.9]",4.0,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
4.2,5.0,2,"[4.0,4.1,4.0]",3.9,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
4.2,3.0,2,[3.9],4.0,109487,Interstellar (2014),Sci-Fi
4.2,4.5,2,"[4.0,4.1,4.0]",4.2,58559,"Dark Knight, The (2008)",Action|Crime|Drama
4.1,4.0,2,"[4.1,4.0,3.9]",4.3,48516,"Departed, The (2006)",Crime|Drama|Thriller
4.1,4.5,2,"[4.0,4.0]",4.1,68157,Inglourious Basterds (2009),Action|Drama
4.1,3.5,2,"[4.0,3.9,3.9]",3.9,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller


In [37]:
def sq_dist(a,b):
    d = np.sum((a-b)**2)
    return d

In [38]:
input_item_m = Input(shape=(num_item_features,))
vm_m = item_NN(input_item_m)
vm_m = Lambda(lambda x: l2_normalize(x, axis=1))(vm_m)
model_m = Model(input_item_m, vm_m)
model_m.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 32)                  │          41,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_2 (Lambda)                    │ (None, 32)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41,376 (161.62 KB)

 Trainable params: 41,376 (161.62 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
size of all predicted movie feature vectors: (847, 32)


In [40]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Wedding Planner, The (2001)",Comedy|Romance,Mr. Deeds (2002),Comedy|Romance
Hannibal (2001),Horror|Thriller,Final Destination 2 (2003),Horror|Thriller
Saving Silverman (Evil Woman) (2001),Comedy|Romance,"Sweetest Thing, The (2002)",Comedy|Romance
Down to Earth (2001),Comedy|Fantasy|Romance,Bewitched (2005),Comedy|Fantasy|Romance
"Mexican, The (2001)",Action|Comedy,Rush Hour 2 (2001),Action|Comedy
15 Minutes (2001),Thriller,Panic Room (2002),Thriller
Enemy at the Gates (2001),Drama,Seabiscuit (2003),Drama
Heartbreakers (2001),Comedy|Crime|Romance,Sex and the City (2008),Comedy|Crime|Romance
Spy Kids (2001),Action|Adventure|Children|Comedy,Scooby-Doo (2002),Action|Adventure|Children|Comedy
